# Train and deploy model on Kubeflow in Notebooks

This examples comes from a upstream fairing [example](https://github.com/kubeflow/fairing/tree/master/examples/prediction).


Please check Kaggle competiton [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)
for details about the ML problem we want to resolve.

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Amazon EKS. This notebook demonstrate how to:

* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow,
* Call the deployed endpoint for predictions.


### Install python dependencies

In [1]:
%%writefile requirements.txt
pandas
joblib
numpy
xgboost
scikit-learn>=0.21.0
seldon-core
tornado>=6.0.3

Overwriting requirements.txt


In [2]:
%%capture
!pip install -r requirements.txt

### Develop your model

In [3]:
import argparse
import logging
import joblib
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

#### Process Data

In [4]:
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

#### Training Function/s

In [5]:
def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds" %
               (model.best_score,
                model.best_iteration+1))
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    logging.info("mean_absolute_error=%.2f", mean_absolute_error(predictions, test_y))

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)
   

#### Hosting Functions

In [6]:
class HousingServe(object):
    
    def __init__(self):
        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names=None):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction

### Train an XGBoost model locally

In [7]:
model = HousingServe()
model.train()

[0]	validation_0-rmse:177565.34375
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161967.20312
[2]	validation_0-rmse:148001.89062
[3]	validation_0-rmse:135010.17188
[4]	validation_0-rmse:123514.68750
[5]	validation_0-rmse:113210.39062
[6]	validation_0-rmse:103914.61719
[7]	validation_0-rmse:95352.96094
[8]	validation_0-rmse:87878.77344
[9]	validation_0-rmse:81683.14062
[10]	validation_0-rmse:75828.78906
[11]	validation_0-rmse:70085.50000
[12]	validation_0-rmse:65076.06641
[13]	validation_0-rmse:60899.83203
[14]	validation_0-rmse:57354.22266
[15]	validation_0-rmse:54106.52734
[16]	validation_0-rmse:51402.42578
[17]	validation_0-rmse:48774.04688
[18]	validation_0-rmse:46360.19141
[19]	validation_0-rmse:44304.82031
[20]	validation_0-rmse:42618.65625
[21]	validation_0-rmse:41219.88672
[22]	validation_0-rmse:39885.14453
[23]	validation_0-rmse:38977.95703
[24]	validation_0-rmse:37856.47656
[25]	validation_0-rmse:36739.78125
[26]	validation_0-rmse:35847

mean_absolute_error=17379.71
Model export success: trained_ames_model.dat


### Create an S3 bucket to store pipeline data
> Note: Be sure to change the HASH variable to random hash before running next cell

> Note: if you use `us-east-1`, please use command `!aws s3 mb s3://{HASH}'-kubeflow-pipeline-data' --region $AWS_REGION --endpoint-url https://s3.us-east-1.amazonaws.com`

In [8]:
import random, string
HASH = ''.join([random.choice(string.ascii_lowercase) for n in range(16)] + [random.choice(string.digits) for n in range(16)])
AWS_REGION = 'us-west-2'
!aws s3 mb s3://{HASH}'-kubeflow-pipeline-data' --region $AWS_REGION

make_bucket: hpqegkputkeotclh5041404659383225-kubeflow-pipeline-data


### Set up Kubeflow Fairing for training and predictions

In [9]:
from kubeflow import fairing
from kubeflow.fairing import TrainJob
from kubeflow.fairing.backends import KubeflowAWSBackend


from kubeflow import fairing

FAIRING_BACKEND = 'KubeflowAWSBackend'

AWS_ACCOUNT_ID = fairing.cloud.aws.guess_account_id()
AWS_REGION = 'us-west-2'
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)
S3_BUCKET = f'{HASH}-kubeflow-pipeline-data'

In [10]:
import importlib

if FAIRING_BACKEND == 'KubeflowAWSBackend':
    from kubeflow.fairing.builders.cluster.s3_context import S3ContextSource
    BuildContext = S3ContextSource(
        aws_account=AWS_ACCOUNT_ID, region=AWS_REGION,
        bucket_name=S3_BUCKET
    )

BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), FAIRING_BACKEND)

### Train an XGBoost model remotely on Kubeflow
Import the `TrainJob` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on Kubeflow.


In [11]:
from kubeflow.fairing import TrainJob
train_job = TrainJob(HousingServe, input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=BackendClass(build_context_source=BuildContext))
train_job.submit()

Using default base docker image: registry.hub.docker.com/library/python:3.6.9
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_q1h_zk7c
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-secret
Waiting for fairing-builder-tqfhc-k2n6f to start...
Waiting for fairing-builder-tqfhc-k2n6f to start...
Waiting for fairing-builder-tqfhc-k2n6f to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0001] Built cross stage deps: map[]
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0003] Unpacking rootfs as cm

  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=427960 sha256=e0ee5817212cf0010d3fbca52e6eefb7151702d626c27c728488a26e007e69b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-nplywkk_/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-cp36-none-any.whl size=9071 sha256=543a62f2d4437cb7e0509e21716f229db773c295f2f7e4de3d9e8b9be041056a
  Stored in directory: /tmp/pip-ephem-wheel-cache-nplywkk_/wheels/7b/dc/25/3cf0b35c129232ee596c413f13d1d1f5a8e38c427266276dfd
  Created wheel for jaeger-client: filename=jaeger_client-4.3.0-cp36-none-any.whl size=64290 sha256=bc0f25143fd1268d18213c1ca55bc2c6fdbed655da366643a94ac7fc031a559e
  Stored in directory: /tmp/pip-ephem-wheel-cache-nplywkk_/wheels/a4/46/e1/316c9ea87f46b9685c2f4258950a27bad1ae072c56224e4489
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=480464 sha256=ba64fa014f9a254abf4e8f87

Not able to find aws credentials secret: aws-secret
The job fairing-job-lz4cx launched.
Waiting for fairing-job-lz4cx-bfz6m to start...
Waiting for fairing-job-lz4cx-bfz6m to start...
Waiting for fairing-job-lz4cx-bfz6m to start...
Pod started running True


[0]	validation_0-rmse:177565.34375
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161967.20312
[2]	validation_0-rmse:148001.89062
[3]	validation_0-rmse:135010.17188
[4]	validation_0-rmse:123514.68750
[5]	validation_0-rmse:113210.39062
[6]	validation_0-rmse:103914.61719
[7]	validation_0-rmse:95352.96094
[8]	validation_0-rmse:87878.77344
[9]	validation_0-rmse:81683.14062
[10]	validation_0-rmse:75828.78906
[11]	validation_0-rmse:70085.50000
[12]	validation_0-rmse:65076.06641
[13]	validation_0-rmse:60899.83203
[14]	validation_0-rmse:57354.22266
[15]	validation_0-rmse:54106.52734
[16]	validation_0-rmse:51402.42578
[17]	validation_0-rmse:48774.04688
[18]	validation_0-rmse:46360.19141
[19]	validation_0-rmse:44304.82031
[20]	validation_0-rmse:42618.65625
[21]	validation_0-rmse:41219.88672
[22]	validation_0-rmse:39885.14453
[23]	validation_0-rmse:38977.95703
[24]	validation_0-rmse:37856.47656
[25]	validation_0-rmse:36739.78125
[26]	validation_0-rmse:35847

Cleaning up job fairing-job-lz4cx...


'fairing-job-lz4cx'

### Deploy the trained model to Kubeflow for predictions

Import the `PredictionEndpoint` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [12]:
from kubeflow.fairing import PredictionEndpoint
endpoint = PredictionEndpoint(HousingServe, input_files=['trained_ames_model.dat', "requirements.txt"],
                              docker_registry=DOCKER_REGISTRY,
                              service_type='ClusterIP',
                              backend=BackendClass(build_context_source=BuildContext))
endpoint.create()

Using default base docker image: registry.hub.docker.com/library/python:3.6.9
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_vy0nz0qs
/usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-secret
Waiting for fairing-builder-rc4q4-h2czd to start...
Waiting for fairing-builder-rc4q4-h2czd to start...
Waiting for fairing-builder-rc4q4-h2czd to start...
Pod started running True


INFO[0005] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0005] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0005] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0006] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0006] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0007] Built cross stage deps: map[]
INFO[0007] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0007] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0007] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0008] Unpacking rootfs as cm

  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=427964 sha256=4e8be1ad0dce1ec9d1d6f3d78fd62ee9494d21166030c7aaf9232a4280519c32
  Stored in directory: /tmp/pip-ephem-wheel-cache-172kdiqc/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793f74d1165f4d0104
  Created wheel for opentracing: filename=opentracing-2.3.0-cp36-none-any.whl size=51348 sha256=329533b5296908caa8b1f80d7c0afa9da1c2b24092e6c7898b3a3a8ba2ec3e77
  Stored in directory: /tmp/pip-ephem-wheel-cache-172kdiqc/wheels/66/b5/60/322949294afda71a9558eb25a1e536d1191665ef71759646b7
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=480450 sha256=8ba29830d35259d5dd8ab07cf259aefa9a3a57a5c59608ccc29f3668ddf9107d
  Stored in directory: /tmp/pip-ephem-wheel-cache-172kdiqc/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for jaeger-client: filename=jaeger_client-4.3.0-cp36-none-any.whl size=64290 sha256=7deaabca56009da33694bf49a4373154d4c

Deploying the endpoint.
Cluster endpoint: http://fairing-service-hq69q.eksworkshop.svc.cluster.local:5000/predict
Prediction endpoint: http://fairing-service-hq69q.eksworkshop.svc.cluster.local:5000/predict


### Call the prediction endpoint
Create a test dataset, then call the endpoint on Kubeflow for predictions.

In [13]:
# Get sample data and query endpoint
(train_X, train_y), (test_X, test_y) = read_input("ames_dataset/train.csv")

# PR https://github.com/kubeflow/fairing/pull/376
# Add `:5000/predict` to mitigate the issue.
endpoint.url='http://fairing-service-hq69q.eksworkshop.svc.cluster.local:5000/predict'

endpoint.predict_nparray(test_X)

{'data': {'names': [],
  'tensor': {'shape': [365],
   'values': [171874.46875,
    107280.5859375,
    177808.359375,
    101154.90625,
    176401.65625,
    51525.8984375,
    119597.921875,
    135324.203125,
    135155.75,
    98387.40625,
    323030.125,
    192741.046875,
    243751.09375,
    176020.53125,
    299734.90625,
    181971.4375,
    197437.875,
    124583.75,
    132495.359375,
    127573.46875,
    310382.125,
    204871.5625,
    136072.703125,
    133508.71875,
    107675.71875,
    111507.3828125,
    214564.375,
    97377.6171875,
    94466.9453125,
    176074.21875,
    121406.2890625,
    196002.59375,
    245223.984375,
    221180.765625,
    133933.015625,
    151592.40625,
    123492.875,
    121962.5546875,
    240069.421875,
    174342.6875,
    105600.515625,
    128300.3984375,
    92803.3515625,
    194406.0,
    123029.171875,
    124194.3046875,
    169212.5,
    386370.5625,
    97869.78125,
    82801.1484375,
    125526.3359375,
    157902.640625,


### Clean up the prediction endpoint
Delete the prediction endpoint created by this notebook.

### Clean up S3 bucket and ECR Repository
Delete S3 bucket and ECR Repository that was created for this exercise

In [16]:
!aws s3 rb s3://$S3_BUCKET --force
!aws ecr delete-repository --repository-name fairing-job --region $AWS_REGION --force

delete: s3://hpqegkputkeotclh5041404659383225-kubeflow-pipeline-data/fairing_builds/2040425D
delete: s3://hpqegkputkeotclh5041404659383225-kubeflow-pipeline-data/fairing_builds/ECC53FF
remove_bucket: hpqegkputkeotclh5041404659383225-kubeflow-pipeline-data
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:500842391574:repository/fairing-job",
        "registryId": "500842391574",
        "repositoryName": "fairing-job",
        "repositoryUri": "500842391574.dkr.ecr.us-west-2.amazonaws.com/fairing-job",
        "createdAt": 1602180103.0
    }
}
